# Minimal Genome Inference

## Introduction

Use comparative genomics to determine which elements of the genomes are highly conserved.

We will assume that all highly conserved elements are essential.

Subset the genome down to these essential sequences, and reconstruct a new path through the genome that will minimize the size to only include these regions of the genome that are inferred essential.

## Methods

Pseudo-algorithm:
- do the following for a virus, mycobacterium, yeast
    - pull list of all genomes on NCBI
    - build pangenome
    - filter out dna sequences with low conservation
    - walk shortest/maximum likelihood path(s)

In [4]:
DATE = "2022-01-15"
TASK = "minimal-genome"
DIR = mkpath("$(homedir())/workspace/$(DATE)-$(TASK)")
cd(DIR)

In [3]:
pkgs = [
"Graphs",
"MetaGraphs",
"BioSequences",
"uCSV",
"DataFrames",
"FASTX",
"HTTP",
"CodecZlib",
"Revise",
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `~/.julia/dev/Mycelia/docs/Project.toml`
  [944b1d66] + CodecZlib v0.7.0
  [626554b9] + MetaGraphs v0.7.1
    Updating `~/.julia/dev/Mycelia/docs/Manifest.toml`
  [e7412a2a] ↑ Ogg_jll v1.3.5+0 ⇒ v1.3.5+1
Precompiling project...
  ✓ StatsAPI
  ✓ Adapt
  ✓ Requires
  ✓ Crayons
  ✓ Static
  ✓ libfdk_aac_jll
  ✓ LAME_jll
  ✓ Wayland_protocols_jll
  ✓ JpegTurbo_jll
  ✓ OpenSSL_jll
  ✓ Ogg_jll
  ✓ OpenSpecFun_jll
  ✓ FriBidi_jll
  ✓ Opus_jll
  ✓ Rmath_jll
  ✓ Kaleido_jll
  ✓ AbstractFFTs
  ✓ IntelOpenMP_jll
  ✓ Graphite2_jll
  ✓ LZO_jll
  ✓ SIMD
  ✓ Xorg_libXdmcp_jll
  ✓ Libuuid_jll
  ✓ x265_jll
  ✓ JuliaInterpreter
  ✓ Libmount_jll
  ✓ Xorg_libXau_jll
  ✓ Zstd_jll
  ✓ StaticArrays
  ✓ Libgpg_error_jll
  ✓ Pixman_jll
  ✓ Xorg_libpthread_stubs_jll
  ✓ Bzip2_jll
  ✓ Libiconv_jll
  ✓ libpng_jll
  ✓ PCRE_jll
  ✓ Xorg_xtrans_jl

### Virus: COVID-19

Find all COVID-19 submissions in NCBI refseq and genbank. Start with refseq

In [28]:
refseq_summary_url = "https://ftp.ncbi.nih.gov/genomes/refseq/assembly_summary_refseq.txt"
refseq_summary_outfile = basename(refseq_summary_url)
if !isfile(refseq_summary_outfile)
    download(refseq_summary_url, refseq_summary_outfile)
end
refseq_summary_table = DataFrames.DataFrame(uCSV.read(refseq_summary_outfile, header=2, delim='\t')...)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCF_000001215.4,PRJNA164,SAMN02803731,,reference genome
2,GCF_000001405.39,PRJNA168,,,reference genome
3,GCF_000001635.27,PRJNA169,,,reference genome
4,GCF_000001735.4,PRJNA116,SAMN03081427,,reference genome
5,GCF_000001905.1,PRJNA70973,SAMN02953622,AAGU00000000.3,representative genome
6,GCF_000001985.1,PRJNA32665,SAMN02953685,ABAR00000000.1,representative genome
7,GCF_000002035.6,PRJNA13922,SAMN06930106,,reference genome
8,GCF_000002075.1,PRJNA209509,SAMN02953658,AASC00000000.3,representative genome
9,GCF_000002235.5,PRJNA13728,SAMN00829422,AAGJ00000000.6,representative genome


Build pangenome graph

Find maximum likelihood path through the genome

### Bacteria: Mycobacterium spp.

Find all COVID-19 submissions in NCBI refseq and genbank. Start with refseq

Build pangenome graph

Find maximum likelihood path through the genome

### Eukaryote: S. cerevisae

Find all COVID-19 submissions in NCBI refseq and genbank. Start with refseq

Build pangenome graph

Find maximum likelihood path through the genome

## Results

## References

https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02130-z

https://www.science.org/doi/10.1126/science.aaf4557

http://syntheticyeast.org/software-development/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3370935/

https://en.wikipedia.org/wiki/Minimal_genome

https://www.science.org/doi/10.1126/science.aad6253

https://www.pnas.org/content/103/2/425